https://medium.com/@ankitvashisht12/classifying-dog-breed-using-pytorch-abc9f3c5128a

In [1]:
from torchvision import transforms
from torch.utils.data import Dataset, random_split, DataLoader
import torch
import torch.nn as nn # layer들을 호출하기 위해서
import numpy as np
import torch.optim as optim # optimization method를 사용하기 위해서
import torch.nn.init as init # weight initialization 해주기 위해서
from tqdm import tqdm

import os
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models

import import_ipynb
from EfficV2_Model import *

from PIL import Image
from collections import OrderedDict
from parallel import DataParallelModel,DataParallelCriterion

# %matplotlib inline

from efficientnet_pytorch import EfficientNet

importing Jupyter notebook from EfficV2_Model.ipynb


In [2]:
batch_size = 64
# num_workers = 2 * torch.cuda.device_count()
num_workers = 64
print(num_workers)
print(os.cpu_count())

64
64


In [3]:
train_set = ImageFolder(root='./stanford/images/train')
test_set = ImageFolder(root='./stanford/images/test')

n_class = len(train_set.classes)

dataset_size = len(train_set)
val_pct = 0.15
val_size = int(dataset_size*val_pct)
train_size = dataset_size - val_size
test_size = len(test_set)

train_size, val_size, test_size, n_class

(8075, 1425, 7058, 95)

In [4]:
train_ds, val_ds = random_split(train_set, [train_size, val_size])
len(train_ds), len(val_ds)

(8075, 1425)

In [5]:
# Image Transform을 지정합니다.
train_transform = transforms.Compose([
#     transforms.Resize((128, 128)),
    transforms.Resize((256, 256)),
    transforms.RandomCrop(256, padding=4, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
#    transforms.Normalize(*imagenet_stats, inplace=True)
])

val_transform = transforms.Compose([
#     transforms.Resize((128, 128)),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
#    transforms.Normalize(*imagenet_stats, inplace=True)
])

test_transform = transforms.Compose([
#     transforms.Resize((128, 128)), 
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
#    transforms.Normalize(*imagenet_stats, inplace=True)
])

In [6]:
# Dataset 로드

train_dataset = Dataset(train_ds, train_transform)
val_dataset = Dataset(val_ds, val_transform)
test_dataset = Dataset(test_set, test_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size, shuffle=True,num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size*2, num_workers=num_workers, pin_memory=True)

In [7]:
# images, classes = next(iter(train_loader))

# images.shape, classes.shape

In [8]:
model = DataParallel(n_class)
# model = PretrainedEfficientNet_V2(n_class)
# model = nn.DataParallel(model, device_ids = [1,2,3,4])
model

Loaded pretrained weights for efficientnet-b4


DataParallel(
  (network): DataParallel(
    (module): EfficientNet(
      (_conv_stem): Conv2dStaticSamePadding(
        3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
        (static_padding): ZeroPad2d((0, 1, 0, 1))
      )
      (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_blocks): ModuleList(
        (0): MBConvBlock(
          (_depthwise_conv): Conv2dStaticSamePadding(
            48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
            (static_padding): ZeroPad2d((1, 1, 1, 1))
          )
          (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (_se_reduce): Conv2dStaticSamePadding(
            48, 12, kernel_size=(1, 1), stride=(1, 1)
            (static_padding): Identity()
          )
          (_se_expand): Conv2dStaticSamePadding(
            12, 48, kernel_size=(1, 1), stride=(1, 1)
            (static_padding): 

In [9]:
# getting default device
# device = torch.device('cuda:1')
device = torch.device('cuda:0')
print(device)

# moving train dataloader and val dataloader to gpu
# train_dl = DeviceDataLoader(train_loader, torch.device('cuda:1'))
train_dl = DeviceDataLoader(train_loader, device)
val_dl = DeviceDataLoader(val_loader, device)


# moving model to gpu
to_device(model, device);

cuda:0


In [10]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
        

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0, grad_clip=None, opt_func = torch.optim.Adam):
    torch.cuda.empty_cache()
    history = []
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # set up one cycle lr scheduler
    #초기 learing rate에서 1cycle annealing하는 scheduler이다. 1주기 전략은 
    #초기 learning rate에서 최대 learning rate까지 올라간 후 초기 learning rate보다 훨씬 낮은 learning rate로 annealing한다. 
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        
        # Training phase
        model.train()       
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            
            # calculates gradients
            loss.backward()
            
            # check gradient clipping 
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
                
            # perform gradient descent and modifies the weights
            optimizer.step()
            
            # reset the gradients
            optimizer.zero_grad()
            
            # record and update lr
            lrs.append(get_lr(optimizer))
            
            # modifies the lr value
            sched.step()
            
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
        
        
    return history

In [11]:
# evaluate(model, val_dl) 

In [12]:
opt_func = torch.optim.SGD
num_epoch = 45
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4

In [ ]:
history = fit_one_cycle(num_epoch, max_lr, model, train_dl, val_dl, weight_decay, grad_clip, opt_func)

 58%|█████▊    | 74/127 [03:28<01:56,  2.20s/it] 

In [ ]:
# num_epoch = 10

# history += fit_one_cycle(num_epoch, max_lr, model, train_dl, val_dl, weight_decay, grad_clip, opt_func)

In [ ]:
# # num_epoch = 5
# # max_lr = 0.001
# # weight_decay = 1e-5

# num_epoch = 10
# max_lr = 0.001
# weight_decay = 1e-3

# history += fit_one_cycle(num_epoch, max_lr, model, train_dl, val_dl, weight_decay, grad_clip, opt_func)

In [ ]:
# num_epoch = 5
# max_lr = 0.01


# history += fit_one_cycle(num_epoch, max_lr, model, train_dl, val_dl, weight_decay, grad_clip, opt_func)

In [ ]:
val_loss = []
train_loss = []
val_acc = []
time = list(range(len(history)))
for h in history:
    val_loss.append(h['val_loss'])
    train_loss.append(h['train_loss'])
    val_acc.append(h['val_acc'])

In [ ]:
plt.plot(time, val_loss, c='red', label='val_loss', marker='x')
plt.plot(time, train_loss, c='blue', label='train_loss', marker='x')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

In [ ]:
plt.plot(time, val_acc, c='red', label='accuracy', marker='x')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [ ]:
lrs = np.concatenate([x.get('lrs', []) for x in history])
plt.xlabel('epochs')
plt.ylabel('lr')
plt.plot(lrs)
plt.show()

In [ ]:
def predict_single(img, label):
    xb = img.unsqueeze(0) # adding extra dimension
    xb = to_device(xb, device)
    preds = model(xb)                   # change model object here
    predictions = preds[0]
    
    max_val, kls = torch.max(predictions, dim=0)
    print('Actual :', breeds[label], ' | Predicted :', breeds[kls])
    plt.imshow(img.permute(1,2,0))
    plt.show()

In [ ]:
breeds = []

def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

for n in train_set.classes:
    breeds.append(rename(n))

In [ ]:

predict_single(*test_dataset[4663])

In [ ]:
test_dl = DeviceDataLoader(test_loader, device)

In [ ]:
result = evaluate(model, test_dl)
result

In [ ]:
# state_dict = model.state_dict()

# for key in list(state_dict.keys()):
#     if 'module.' in key:
#         new_key = key.replace('module.', '') # DataParallel 사용 시 key 값이 바뀜
#         state_dict[new_key] = state_dict[key]
#         del state_dict[key]

In [ ]:
# torch.save(model, './models/dataloader-2_EfficV2_4.pt')

In [ ]:
torch.save(model.network.module.state_dict(), './model_network/9/model_network_module_dict.pt')

In [ ]:
torch.save(model.network.state_dict(), './model_network/9/model_network_dict.pt')

In [ ]:
torch.save(model.module.state_dict(), './model_network/9/model_module_dict.pt')

In [ ]:
torch.save(model.state_dict(), './model_network/9/model_dict.pt')